In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from implementations import * 
from utilities.logistic_regression import * #todo specify with the ones we use
from utilities.linear_regression import * #todo specify with the ones we use
from utilities.cross_validation import * #todo specify with the ones we use

In [ ]:
# Load the dataset
x_tr_original, x_te_original, y_tr, tr_id, te_id = load_csv_data('dataset', False)
# Transform our y values from {-1,1} to {0,1} because that’s what logistic regression tests expect https://edstem.org/eu/courses/1605/discussion/134447
y_tr[y_tr == -1] = 0

In [ ]:
# Make a copy so we can just rerun this cell to start over instead of re-loading the dataset from disk, which is long
x_tr = x_tr_original#.copy()
x_te = x_te_original#.copy()

### Data exploration and data cleaning

In [ ]:
"""
We analyze the percentage of NaN values in each feature to determine an appropriate cutoff threshold.
Features with too many NaN values are less useful, but we don’t want to remove too many features.

number of features left for each threshold value:
   5% nan: 115 -> 35% of features left
  50% nan: 174 -> 54%
  65% nan: 194 -> 60%
  90% nan: 222 -> 69%
we pick a threshold of 50% for now.
TODO: test difference in accuracy with different thresholds {5, 50, 65, 90}%
"""

def filter_features_with_too_many_nans(x_tr, x_te):
    # Count the percentage of NaN values per feature
    nan_percentage_by_feature = (np.sum(np.isnan(x_tr), axis=0) / len(x_tr)) * 100
    
    # Plot the resulting distribution
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 4))
    plt.hist(nan_percentage_by_feature, bins=100)
    plt.title("Distribution of NaN Percentage by feature")
    plt.xlabel("NaN Percentage in the feature")
    plt.ylabel("Number of features")
    plt.xticks(np.arange(start=0, stop=101, step=5))
    
    # A threshold of maximum 50% of NaN values in a feature seems like a reasonable tradeoff
    MAX_NAN_PERCENTAGE = 50
    mask = nan_percentage_by_feature < MAX_NAN_PERCENTAGE
    # So we get rid of the features with more than MAX_NAN_PERCENTAGE % of NaN values
    x_tr = x_tr.copy()[:, mask]
    x_te = x_te.copy()[:, mask]
    from_nan_filtered_to_original_features = np.where(mask)[0]
    return x_tr, x_te, from_nan_filtered_to_original_features

x_tr, x_te, from_nan_filtered_to_original_features = filter_features_with_too_many_nans(x_tr, x_te)

In [ ]:
# Our objective is to build a mask to separate numerical and co (categorical/ordinal) data. 
# First we explore the data to see what is the maximum number of unique values for a co feature.

# Compute and plot the distribution of unique values 
unique_counts_and_indices = np.array([[len(np.unique(x_tr[:, i])),i] for i in range(x_tr.shape[1])])
unique_counts = unique_counts_and_indices[:,0]

plt.hist(unique_counts, bins=50) # Naive visualization, just to have an idea
plt.xlabel('Number of Unique Values')
plt.ylabel('Frequency')
plt.title('Distribution of Unique Values Across Features')
plt.show()

# We consider that a feature with less than 10 values can be considered categorical/ordinal (co),
# and one with more than distinct 60 values is surely numerical (think about a human having to
# pick 100 options in a survey)

# Our goal is to find the co feature with the most distinct values to use it as a threshold
unique_counts_and_indices_refined = unique_counts_and_indices[(unique_counts_and_indices[:,0] > 10) & (unique_counts_and_indices[:,0] < 60)]
# We get back the original indexes of our x_tr to link our findings to the original feature indices
indices_to_consider = from_nan_filtered_to_original_features[unique_counts_and_indices_refined[:,1]]

In [ ]:
x_label = np.array(["_STATE", "FMONTH", "IDATE", "IMONTH", "IDAY", "IYEAR", "DISPCODE", "SEQNO", "_PSU", "CTELENUM", "PVTRESD1", "COLGHOUS", "STATERES", "CELLFON3", "LADULT", "NUMADULT", "NUMMEN", "NUMWOMEN", "CTELNUM1", "CELLFON2", "CADULT", "PVTRESD2", "CCLGHOUS", "CSTATE", "LANDLINE", "HHADULT", "GENHLTH", "PHYSHLTH", "MENTHLTH", "POORHLTH", "HLTHPLN1", "PERSDOC2", "MEDCOST", "CHECKUP1", "BPHIGH4", "BPMEDS", "BLOODCHO", "CHOLCHK", "TOLDHI2", "CVDSTRK3", "ASTHMA3", "ASTHNOW", "CHCSCNCR", "CHCOCNCR", "CHCCOPD1", "HAVARTH3", "ADDEPEV2", "CHCKIDNY", "DIABETE3", "DIABAGE2", "SEX", "MARITAL", "EDUCA", "RENTHOM1", "NUMHHOL2", "NUMPHON2", "CPDEMO1", "VETERAN3", "EMPLOY1", "CHILDREN", "INCOME2", "INTERNET", "WEIGHT2", "HEIGHT3", "PREGNANT", "QLACTLM2", "USEEQUIP", "BLIND", "DECIDE", "DIFFWALK", "DIFFDRES", "DIFFALON", "SMOKE100", "SMOKDAY2", "STOPSMK2", "LASTSMK2", "USENOW3", "ALCDAY5", "AVEDRNK2", "DRNK3GE5", "MAXDRNKS", "FRUITJU1", "FRUIT1", "FVBEANS", "FVGREEN", "FVORANG", "VEGETAB1", "EXERANY2", "EXRACT11", "EXEROFT1", "EXERHMM1", "EXRACT21", "EXEROFT2", "EXERHMM2", "STRENGTH", "LMTJOIN3", "ARTHDIS2", "ARTHSOCL", "JOINPAIN", "SEATBELT", "FLUSHOT6", "FLSHTMY2", "IMFVPLAC", "PNEUVAC3", "HIVTST6", "HIVTSTD3", "WHRTST10", "PDIABTST", "PREDIAB1", "INSULIN", "BLDSUGAR", "FEETCHK2", "DOCTDIAB", "CHKHEMO3", "FEETCHK", "EYEEXAM", "DIABEYE", "DIABEDU", "CAREGIV1", "CRGVREL1", "CRGVLNG1", "CRGVHRS1", "CRGVPRB1", "CRGVPERS", "CRGVHOUS", "CRGVMST2", "CRGVEXPT", "VIDFCLT2", "VIREDIF3", "VIPRFVS2", "VINOCRE2", "VIEYEXM2", "VIINSUR2", "VICTRCT4", "VIGLUMA2", "VIMACDG2", "CIMEMLOS", "CDHOUSE", "CDASSIST", "CDHELP", "CDSOCIAL", "CDDISCUS", "WTCHSALT", "LONGWTCH", "DRADVISE", "ASTHMAGE", "ASATTACK", "ASERVIST", "ASDRVIST", "ASRCHKUP", "ASACTLIM", "ASYMPTOM", "ASNOSLEP", "ASTHMED3", "ASINHALR", "HAREHAB1", "STREHAB1", "CVDASPRN", "ASPUNSAF", "RLIVPAIN", "RDUCHART", "RDUCSTRK", "ARTTODAY", "ARTHWGT", "ARTHEXER", "ARTHEDU", "TETANUS", "HPVADVC2", "HPVADSHT", "SHINGLE2", "HADMAM", "HOWLONG", "HADPAP2", "LASTPAP2", "HPVTEST", "HPLSTTST", "HADHYST2", "PROFEXAM", "LENGEXAM", "BLDSTOOL", "LSTBLDS3", "HADSIGM3", "HADSGCO1", "LASTSIG3", "PCPSAAD2", "PCPSADI1", "PCPSARE1", "PSATEST1", "PSATIME", "PCPSARS1", "PCPSADE1", "PCDMDECN", "SCNTMNY1", "SCNTMEL1", "SCNTPAID", "SCNTWRK1", "SCNTLPAD", "SCNTLWK1", "SXORIENT", "TRNSGNDR", "RCSGENDR", "RCSRLTN2", "CASTHDX2", "CASTHNO2", "EMTSUPRT", "LSATISFY", "ADPLEASR", "ADDOWN", "ADSLEEP", "ADENERGY", "ADEAT1", "ADFAIL", "ADTHINK", "ADMOVE", "MISTMNT", "ADANXEV", "QSTVER", "QSTLANG", "MSCODE", "_STSTR", "_STRWT", "_RAWRAKE", "_WT2RAKE", "_CHISPNC", "_CRACE1", "_CPRACE", "_CLLCPWT", "_DUALUSE", "_DUALCOR", "_LLCPWT", "_RFHLTH", "_HCVU651", "_RFHYPE5", "_CHOLCHK", "_RFCHOL", "_LTASTH1", "_CASTHM1", "_ASTHMS1", "_DRDXAR1", "_PRACE1", "_MRACE1", "_HISPANC", "_RACE", "_RACEG21", "_RACEGR3", "_RACE_G1", "_AGEG5YR", "_AGE65YR", "_AGE80", "_AGE_G", "HTIN4", "HTM4", "WTKG3", "_BMI5", "_BMI5CAT", "_RFBMI5", "_CHLDCNT", "_EDUCAG", "_INCOMG", "_SMOKER3", "_RFSMOK3", "DRNKANY5", "DROCDY3_", "_RFBING5", "_DRNKWEK", "_RFDRHV5", "FTJUDA1_", "FRUTDA1_", "BEANDAY_", "GRENDAY_", "ORNGDAY_", "VEGEDA1_", "_MISFRTN", "_MISVEGN", "_FRTRESP", "_VEGRESP", "_FRUTSUM", "_VEGESUM", "_FRTLT1", "_VEGLT1", "_FRT16", "_VEG23", "_FRUITEX", "_VEGETEX", "_TOTINDA", "METVL11_", "METVL21_", "MAXVO2_", "FC60_", "ACTIN11_", "ACTIN21_", "PADUR1_", "PADUR2_", "PAFREQ1_", "PAFREQ2_", "_MINAC11", "_MINAC21", "STRFREQ_", "PAMISS1_", "PAMIN11_", "PAMIN21_", "PA1MIN_", "PAVIG11_", "PAVIG21_", "PA1VIGM_", "_PACAT1", "_PAINDX1", "_PA150R2", "_PA300R2", "_PA30021", "_PASTRNG", "_PAREC1", "_PASTAE1", "_LMTACT1", "_LMTWRK1", "_LMTSCL1", "_RFSEAT2", "_RFSEAT3", "_FLSHOT6", "_PNEUMO2", "_AIDTST3"])
x_label[indices_to_consider]

```
co = categorical/ordinal, n = numerical
_STATE: 53 co
FMONTH: 12 
IMONTH: 12 
IDAY: 31 
PHYSHLTH: 32 
MENTHLTH: 32 
POORHLTH: 32 
CHILDREN: 11
INCOME2: 11
HEIGHT3: 57 n
ALCDAY5: 38
FRUITJU1: 50 
FVBEANS: 49
FVGREEN: 56 wtf is this format 😂
FVORANG: 52
STRENGTH: 49
_AGEG5YR: 14
HTIN4: 32
HTM4: 46
DROCDY3_: 34
FTJUDA1_: 43
FRUTDA1_: 59 n
BEANDAY_: 42
GRENDAY_: 48
ORNGDAY_: 43
VEGEDA1_: 58 I honestly dont know
METVL11_: 29
METVL21_: 28
PAFREQ1_: 55
STRFREQ_: 44
```

--> The answer was right under our nose ! The co feature with the most distinct values is _STATE.

After methodical data exploration and analysis, we observed that the co feature that had 
the most distinct values (53) was _STATE which is our first feature. 

We consider every feature that has 53 distinct values or less as categorical or ordinal, and every feature that has 54 values or more as numerical.
 
TODO: this is with the NAN THRESHOLD = 50 -> update if we change threshold.

In [ ]:
THRESHOLD_CO = 53
index_of_co_features = np.where(unique_counts_and_indices[:,0] <= THRESHOLD_CO)[0]
index_of_numerical_features = np.where(unique_counts_and_indices[:,0] > THRESHOLD_CO)[0]

Now that we have isolated co features from numerical features, we can
- Impute missing values represented by a nan correctly (ex: replace nan with mode if co, replace nan with mean if numerical)
- Encode correctly our co features by:
    - One hot encoding
    - Keeping the original feature to make sure we dont use the ordinality of the ordinal features, as we dont make a difference between ordinal and categorical features.

TODO: manual cleaning to get rid of residual nan (for example: 9999 values that still mean nan). Maybe we get rid of them when we get rid of outliers later ?

In [ ]:
'''
Starting here, we assume that every unusable value is a nan.

How do we impute missing values ?
We can take two routes:
1) Univariate imputation of missing values (we only consider the values of the related feature to impute missing values)
    - For numerical features: We replace by the mean or the median 
    - For co features: We replace nan values by a unique value to distinguish them
2) Multivariate imputation of missing values (we take into account all of the features to impute missing values)
    - Multiple techniques exist (KNN based, regression based...)


You have to agree that option 2 has a lot more style (and relevance). But it could also be a pain to implement with numpy 
(one usually uses scikit learn for this purpose) and as we did not see anything in class, 
all that we do is extra so we will go the easy route (1) for now. 

todo can we try 2) Multivariate imputation
'''

def impute_nan_values(x_tr, x_te, num_indices, co_indices):
    # For numerical features: replace nan with median (more robust to outliers)
    medians = np.nanmedian(x_tr[:, num_indices], axis=0)
    x_tr[:, num_indices] = np.nan_to_num(x_tr[:, num_indices], nan=medians)
    x_te[:, num_indices] = np.nan_to_num(x_te[:, num_indices], nan=medians)
    
    # For categorical/ordinal features: replace nan with maximum + 1
    # It acts as a kind of new “nan category,” which we believe makes
    # more sense than taking the feature’s mode
    maximums = np.nanmax(x_tr[:, co_indices], axis=0) + 1
    x_tr[:, index_of_co_features] = np.nan_to_num(x_tr[:, co_indices], nan=maximums)
    x_te[:, index_of_co_features] = np.nan_to_num(x_te[:, co_indices], nan=maximums)
    
    return x_tr, x_te

x_tr, x_te = impute_nan_values(x_tr, x_te, index_of_numerical_features, index_of_co_features)

In [ ]:
'''
Feature transformation

- For numerical features: We standardize by substracting the mean and dividing by the std (todo we can also normalize them, which is better ? (ex: search min max normalization))
- For co features: 
    1) We add the one hot encoding while also keeping the original feature (to keep relations of ordinality for ordinal features)
    2) Can we handle nan values this way ? We need to check the results. (todo give me your opinion) 
'''

def standardize_some_features(x_tr, x_te, features_mask):
    means = np.mean(x_tr[:, features_mask], axis=0)
    stds = np.std(x_tr[:, features_mask], axis=0)
    stds = np.where(stds == 0, 1, stds)
    x_tr[:, features_mask] = (x_tr[:, features_mask] - means) / stds
    x_te[:, features_mask] = (x_te[:, features_mask] - means) / stds
    return x_tr, x_te

x_tr, x_te = standardize_some_features(x_tr, x_te, index_of_numerical_features)

def encode_categorical_ordinal_features(x_tr, x_te):
    merged_x = np.vstack((x_tr, x_te))
    result = []
    for x in [x_tr, x_te]:
        N = len(x)
        for idx in index_of_co_features:
            unique_values = np.unique(merged_x[:, idx])
            one_hot_length = len(unique_values)
            one_hot_features = np.zeros((N, one_hot_length)) #todo say dtype is int ?
        
            # Here the objective is to go from the value to the index in the array of unique values
            val_to_index = {value: index for index, value in enumerate(unique_values)}
            
            for i in range(N):
                val = x[i, idx]
                val_index = val_to_index[val]
                one_hot_features[i, val_index] = 1
        
            # Add these new features to our x matrix
            x = np.hstack((x, one_hot_features))
    
        print(x.shape)
        result.append(x)

    # We also standardize the original co features which can have funky values
    x_tr, x_te = standardize_some_features(result[0], result[1], index_of_co_features)
    
    return x_tr, x_te
    
x_tr, x_te = encode_categorical_ordinal_features(x_tr, x_te)

In [ ]:
# We are now expecting no nan in our dataset whatsoever.
assert not np.isnan(x_tr).any()


that are not possible according to the documentation through boxplots or condition enforcement 
(schematic example: enforce that weights are inferior to 1000 and if not cap them to the max(1000)) 
we can also z-score normalization and check outside [-3,3] which indicates outlier (understand method). We need to make it as automatic as possible, this seems like a better idea.

In [ ]:
'''
Handling outliers on numerical features

We define percentiles to floor and cap the numerical data in order to clean our data from unexpected outliers.
'''

def clip_numerical_features(x_tr, x_te, indices_num_feat, floor_p=10, cap_p=90):
    """
    Args:
        x_tr: numpy array of shape (N,D), D is the number of features.
        floor_p: percentile on which we floor the numerical features.
        cap_p: percentile on which we cap the numerical features.
        indices_num_feat: The indices of x_tr numerical features.
    Returns:
        x_tr: numpy array of shape (N,D), D is the number of features. Outliers have been clipped.
        x_te: numpy array of shape (N_test,D), D is the number of features. Outliers have been clipped.
    """
    floors = np.percentile(x_tr[:,indices_num_feat], floor_p, axis=0)
    caps = np.percentile(x_tr[:,indices_num_feat], cap_p, axis=0)
    x_tr[:, indices_num_feat] = np.clip(x_tr[:, indices_num_feat], floors, caps)
    x_te[:, indices_num_feat] = np.clip(x_te[:, indices_num_feat], floors, caps)
    return x_tr, x_te

x_tr, x_te = clip_numerical_features(x_tr, x_te, index_of_numerical_features)

# Outliers have been clipped and there is no Z-score with an absolute value higher than 3
assert not (np.abs(x_tr[:,index_of_numerical_features]) > 3).any()

### Feature Selection

Using a correlation matrix, we would capture pairwise correlation relationships but our dataset captures correlations involving the interaction between three or more variables (ex: dates) 
      

We can use another approach: We do a regularized logistic regression, which forces the model to give more weight to highly predictive features, and close to 0 weight to useless features (as the model tries to spare the amount of weight).

We train our model using all the features a first time, then compare the absolute values of the weights and get rid of the features that have an associated weight that has a low enough absolute value (they were not deemed predictive during the regularized regression)

As we have seen in ADA, the lower the absolute value of a weight, the less impact a change in the related input feature will have on the prediction.

In [ ]:
def build_tx(x):
    return np.c_[np.ones((x.shape[0], 1)), x]

In [ ]:
def test_model(w, tx_test, y_test):
    # Sigmoid gives a value between 0 and 1
    test_probs = sigmoid(tx_test @ w)
    # We round to the nearest to get our prediction in {0,1}
    # (which we will transform later into {-1,1} for submission
    test_preds = np.round(test_probs)
    test_error_rate = np.count_nonzero(y_test - test_preds) / len(y_test)
    test_accuracy = 1 - test_error_rate
    test_f1 = compute_f1_score(y_test, test_preds)

    return test_f1, test_accuracy

In [ ]:
def plot_scores_against_features_kept(scores, percent_of_features_kept):
    for score_name, score_values in scores.items():
        plt.plot(percent_of_features_kept, score_values, marker='o')
        plt.xlabel("Percentage of features kept")
        plt.ylabel(score_name)
        plt.title(f"{score_name} as we keep more or less features")
        plt.xticks(percent_of_features_kept)
        plt.grid(True)
        plt.show()

In [ ]:
def train_initial_model(y_tr, tx_train, lambda_, max_iter, gamma):
    initial_w = np.zeros(tx_train.shape[1])
    w, loss = reg_logistic_regression(y_tr, tx_train, lambda_, initial_w, max_iter, gamma)

    # Plot the magnitude of the learned weights
    # We see that lots of features carry little weight in the predictions
    # So we will try removing the least important features
    w_abs = np.abs(w)
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(w_abs)), w_abs)
    plt.xlabel("Index")
    plt.ylabel("Value of w abs")
    plt.title("Values in vector w")
    plt.show()
    
    return w_abs

In [ ]:
# We want to simplify the model, get better performance and reduce overfitting risk,
# by removing the features that do not contribute much to the predictions.

# So we only select the top P percent of features that contribute the most to the
# predictions, judging by the weight they have in the weight vector w we learned with
# logistic regression.

# The percentile P is the minimum weight value we require to keep P percent of the features.
# We select this P methodically by testing a wide range of possible such thresholds.


def split_train_data(x_tr, y_tr, train_percentage=0.9):
    """
    We only need this to find the best features to keep.
    For the hyperparameter selection we have proper cross-validation.
    """
    random_seed = 0 #TODO: use random seed
    np.random.seed(random_seed)

    N = x_tr.shape[0]
    indices = np.random.permutation(N)

    tr_set_size = int(N * train_percentage)
    tr_indices = indices[:tr_set_size]
    te_indices = indices[tr_set_size:]

    x_tr = x_tr.copy()
    X_train, X_test = x_tr[tr_indices], x_tr[te_indices]
    y_train, y_test = y_tr[tr_indices], y_tr[te_indices]

    return X_train, X_test, y_train, y_test

def select_most_significant_features():
    # Split the data so we can compute scores on a test set for each selection of features
    X_train, X_test, y_train, y_test = split_train_data(x_tr, y_tr, train_percentage=0.9)
    tx_train = build_tx(X_train)
    tx_test = build_tx(X_test)

    # We saw that a small lambda is better to allow for more variance in
    # weights, which makes picking the most important features easier
    max_iter = 100
    gamma = 0.5
    lambda_ = 0.0005

    # Train a first model using all the features
    w_abs = train_initial_model(y_train, tx_train, lambda_, max_iter, gamma)

    # We want to test keeping between 100% and 5% of the most significant features
    # This means having a threshold weight between the 0th and the 95th percentile of weight values
    percentiles_to_test = np.arange(0, 100, 5)
    threshold_w = [np.percentile(w_abs, p) for p in percentiles_to_test]
    scores = {
        'Test F1 Score': [],
        'Test Accuracy': [],
        'Train Loss': []
    }
    best_f1 = 0.0
    best_features_to_keep = None
    for t in threshold_w:
        features_to_keep = w_abs > t
        tx_train_filtered = tx_train[:, features_to_keep]
        tx_test_filtered = tx_test[:, features_to_keep]
        initial_w = np.zeros(tx_train_filtered.shape[1])
    
        w, train_loss = reg_logistic_regression(y_train, tx_train_filtered, lambda_, initial_w, max_iter, gamma)
    
        test_f1, test_accuracy = test_model(w, tx_test_filtered, y_test)
    
        # Store the results
        scores['Test F1 Score'].append(test_f1)
        scores['Test Accuracy'].append(test_accuracy)
        scores['Train Loss'].append(train_loss)
        # Store the feature selection if this gives the best f1 score so far
        if test_f1 > best_f1:
            best_features_to_keep = features_to_keep

    # We kept the top x% of features contributing most to predictions
    percent_of_features_kept = 100 - percentiles_to_test
    # Plot the result of our tests
    plot_scores_against_features_kept(scores, percent_of_features_kept)
    # Return the feature selection that gave the best F1 score
    return best_features_to_keep

best_features_to_keep = select_most_significant_features()

# 3. Cross validation and model selection

In [ ]:
lambdas = 5 * 10 ** (-np.arange(5, 9, dtype=np.float64))
tx_train = build_tx(x_tr)[:, best_features_to_keep]
initial_w = np.zeros(tx_train.shape[1])
max_its = 50 * np.arange(1, 21)
gammas = [1, 1.3, 1.5]
fold_nums = 5
# WARNING: Takes a LOT of time (6 min)
best_f1, best_hps, scores = cross_validation_for_parameter_selection(
    y_tr, tx_train, kind='logistic', fold_nums=fold_nums, verbose=True, lambdas=lambdas, initial_w=initial_w, max_its=max_its, gammas=gammas
)

In [ ]:
print("Best f1 score: {:.2f}\nBest hyperparameters:\n\t{}".format(best_f1, "\n\t".join(["{:10s} {}".format(k, v) for k, v in best_hps.items()])))

In [ ]:
fig, axs = plt.subplots(nrows=len(lambdas), ncols=len(gammas), sharex=True, sharey=True)
fig.set_size_inches(14, 12)
fig.suptitle("Regularized Logistic Regression:\naccuracy and F1 score for each combination of hyperparameters", fontsize='xx-large')
for ilambda_, lambda_ in enumerate(lambdas):
    for igamma, gamma in enumerate(gammas):
        ll = scores[ilambda_][igamma]
        ax = axs[ilambda_][igamma]
        ax.plot(max_its, ll['accuracy'], '.-', label='Accuracy')
        ax.plot(max_its, ll['f1'], '.-', label='F1 Score')
        ax.set_xticks(max_its, [str(m) if i % 4 == 1 else "" for i, m in enumerate(max_its)], rotation='vertical')
        ax.set_ybound(-0.05, 1)
        yticks = np.linspace(0, 1, 11)
        ax.set_yticks(yticks, ["{:.1f}".format(x) if i % 2 == 0 else "" for i, x in enumerate(yticks)])
        if igamma == len(gammas) - 1:
            ax.yaxis.set_label_coords(1, 0.5)
            ax.set_ylabel("lambda = {:.0e}".format(lambda_), rotation=270, fontsize='large')
        if ilambda_ == 0:
            ax.set_title(f"gamma = {gamma}")
            if igamma == len(gammas) - 1:
                ax.legend(loc='upper right', ncols=2, bbox_to_anchor=(1, 1.3))
        if ilambda_ == len(lambdas) - 1:
            ax.set_xlabel("Iterations")
fig.savefig('cross_validation')

# 4. Train the final model and generate predictions for test.csv

In [ ]:
# Load the dataset, without sub-sampling
x_tr_full_original, x_te_full_original, y_tr_full, tr_id_full, te_id_full = load_csv_data('dataset', sub_sample=False)
# Transform our y values from {-1,1} to {0,1} because that’s what logistic regression tests expect https://edstem.org/eu/courses/1605/discussion/134447
y_tr_full[y_tr_full == -1] = 0

In [ ]:
# Make a copy so we can just rerun this cell to start over instead of re-loading the dataset from disk, which is long
x_tr_full = x_tr_full_original#.copy()
x_te_full = x_te_full_original#.copy()

In [ ]:
# Preprocess the data
x_tr_full, x_te_full, _ = filter_features_with_too_many_nans(x_tr_full, x_te_full)
x_tr_full, x_te_full = impute_nan_values(x_tr_full, x_te_full, index_of_numerical_features, index_of_co_features)
x_tr_full, x_te_full = standardize_some_features(x_tr_full, x_te_full, index_of_numerical_features)
x_tr_full, x_te_full = encode_categorical_ordinal_features(x_tr_full, x_te_full)
x_tr_full, x_te_full = clip_numerical_features(x_tr_full, x_te_full, index_of_numerical_features)
tx_train_full = build_tx(x_tr_full)[:, best_features_to_keep]

In [ ]:
# Train the final model with the best hyperparameters and the best feature selection
initial_w = np.zeros(tx_train_full.shape[1])
final_w, final_loss = reg_logistic_regression(y_tr_full, tx_train_full, best_hps.lambda_, initial_w, 100, best_hps.gamma)

In [ ]:
# Generate predictions and save them to .csv

tx_test_full = build_tx(x_te_full)[:, best_features_to_keep]
# Sigmoid gives a value between 0 and 1
test_probs_full = sigmoid(tx_test_full @ final_w)
# We round to the nearest to get our prediction in {0,1}
test_preds_full = np.round(test_probs_full)
# Transform them into {-1,1} for submission
test_preds_full[test_preds_full == 0] = -1
create_csv_submission(te_id_full, test_preds_full, 'full_test_preds.csv')